In [73]:
import numpy as np
import pandas as pd

In [74]:
df = pd.read_csv('../data/userData.csv')

df.drop(['weekday'], axis=1, inplace=True)
df

,food_name,time_of_day
0,chipotle chicken and queso bowl,Morning
1,kodiak cake waffles,Night
2,chipotle bowl chicken and queso,Evening
3,starbucks caramel macchiato,Morning
4,chicken tacos,Evening
...,...,...
374,mcdonald’s cheeseburger,Evening
375,small flour tortillas,Evening
376,chicken,Evening
377,2 percent fairlife chocolate milk,Evening


In [75]:
from sklearn.preprocessing import LabelEncoder

data_encoded = pd.get_dummies(df[['time_of_day']]) # time of day is Night, Morning, Afternoon, Evening
data_encoded = pd.concat([df[['food_name']], data_encoded], axis=1) # food_name is a string 

y = df['food_name'].values

encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)

data_encoded['encoded_y'] = encoded_y

data_encoded.sort_values('food_name', inplace=True)

data_encoded

,food_name,time_of_day_Afternoon,time_of_day_Evening,time_of_day_Morning,time_of_day_Night,encoded_y
250,12 count meal from chick-fil-a,False,True,False,False,0
377,2 percent fairlife chocolate milk,False,True,False,False,1
227,2 pieces of toast with jelly and butter,False,True,False,False,2
261,3 chocolate doughnuts,True,False,False,False,3
34,3x3 from in-n-out,False,True,False,False,4
...,...,...,...,...,...,...
135,whataburger fries,True,False,False,False,144
133,whataburger sweet tea,True,False,False,False,145
80,yogurt,False,True,False,False,146
303,yogurt,False,True,False,False,146


In [76]:
from sklearn.model_selection import train_test_split

import tensorflow as tf

In [77]:
top_n = 5
y = data_encoded['encoded_y'].values
X = data_encoded.drop(['food_name', 'encoded_y'], axis=1).values

print(tf.__version__)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)
X_train = X
y_train = y

num_classes = len(np.unique(y_train))
print(num_classes, len(y_train)) # 139


y_train = tf.keras.utils.to_categorical(y_train, num_classes)


2.16.1
147 379


In [78]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(4,), name='input'),
    tf.keras.layers.Dense(64, activation='relu', name='hidden1'),
    tf.keras.layers.Dense(32, activation='relu', name='hidden2'),
    tf.keras.layers.Dense(num_classes, activation='softmax', name='output')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[tf.keras.metrics.TopKCategoricalAccuracy(k=top_n)])

/usr/local/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [79]:
# model.fit(X_train, y_train, batch_size=32, epochs=40, validation_data=(X_test, y_test))
model.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 4.9678 - top_k_categorical_accuracy: 0.1627
Epoch 2/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.8775 - top_k_categorical_accuracy: 0.4264 
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.7217 - top_k_categorical_accuracy: 0.4213 
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.4442 - top_k_categorical_accuracy: 0.4100 
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.1235 - top_k_categorical_accuracy: 0.3782 
Epoch 6/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.9820 - top_k_categorical_accuracy: 0.4174 
Epoch 7/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.9281 - top_k_categorical_accuracy: 0.4090 
Epoch 8/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.9039 - top_k_categorical_accuracy: 0.4189 
Epoch 9/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.9257 - top_k_categorical_accuracy: 0.4086 
Epoch 10/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.785

In [80]:
permutations_day = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]], dtype=np.bool_)


predictions = model.predict(permutations_day)



top_n_indices = np.argsort(predictions, axis=1)[:, -top_n:][:, ::-1]
top_n_probabilities = np.sort(predictions, axis=1)[:, -top_n:][:, ::-1]


top_n_indices, top_n_probabilities

flattened =  encoder.inverse_transform(top_n_indices.flatten())

top_n_labels = flattened.reshape(top_n_indices.shape)

top_n_labels, top_n_probabilities

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 377ms/step


(array([['chick-fil-a sandwich', 'sweet tea', 'kodiak cake waffles',
         'borden chocolate milk', 'milk'],
        ['chick-fil-a sandwich', 'sweet tea',
         'chick-fil-a granola parfait', 'banana', 'milk'],
        ['protein shake with milk and peanut butter', 'cups of milk',
         'chipotle chicken and queso bowl', 'cheddar cheese',
         'optimum nutrition shake'],
        ['kodiak cake waffles', 'orange juice', 'cashews', '4oz chicken',
         'chobani yogurt']], dtype=object),
 array([[0.10047881, 0.09818518, 0.03401121, 0.03302214, 0.02372774],
        [0.2578329 , 0.2156509 , 0.02628623, 0.02254894, 0.01572021],
        [0.10409047, 0.09469175, 0.09428594, 0.05673443, 0.05146712],
        [0.13636449, 0.07835032, 0.07773016, 0.07706612, 0.07505353]],
       dtype=float32))

In [81]:
time_of_day_categories = ['Morning', 'Afternoon', 'Evening', 'Night']

with open('../out/time_based_output_tensorflow_network.txt', 'w') as f:
    # Iterate over time of day categories
    for time_of_day, predictions, probabilities in zip(time_of_day_categories, top_n_labels, top_n_probabilities):
        # Write time of day
        f.write("Time of Day: {}\n".format(time_of_day))
        f.write("Predictions:\n")
        # Iterate over predictions and probabilities
        for i, (prediction, probability) in enumerate(zip(predictions, probabilities), 1):
            # Write each prediction and its probability
            f.write("    - Prediction {}: {}, Probability: {:.2f}%\n".format(i, prediction, probability * 100))
        # Add a newline for better readability between time of day categories
        f.write("\n")
